In [1]:
import os

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# <font color='red'>**Data loader**</font>
## **Useful libraries**

In [3]:
import scipy
from glob import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

In [4]:
#TALE SECOND PART:
class DataLoader():      
    
    """ Data loader method: loader initialization on training or test batch

    Parameters
    -------------------   
    dataset_name: string- dataset name
    img_res: array- image shape (n_rows, n_cols,n_channels)
    path_csv: string- path csv (que contiene el csv?, es importante?***)
    use_test_in_batch: boolean- decides if the data is test or not
    normalize: boolean- for image normalization
    """
    #csv dataset has three columns: Emotion: (label from 0 to 6)
                              #   Pixels: (pixel values from 48X48 image)
                              #   Usage: split to be used (train or test) 
    def __init__(self, dataset_name, img_res=(48, 48,1), path_csv=None, use_test_in_batch=False, 
                 normalize=True):
        
        self.dataset_name = dataset_name
        self.img_res = img_res
        # images and labels vectors for train and test 
        self.img_vect_train = None 
        self.img_vect_test = None 
        self.lab_vect_train = None 
        self.lab_vect_test = None 
        self.path_csv = path_csv 
        ## labels dict
        self.lab_dict = {0: "Angry", 1: "Disgust" , 2: "Fear" , 3: "Happy" , 4: "Sad" , 
                        5: "Surprise" , 6: "Neutral"}
        self.use_test_in_batch = use_test_in_batch
        self.normalize = normalize 
        ## load dataset 
        self._load_internally()


    def _load_internally(self):

        """

        """

        print(">> loading "+str(self.dataset_name)+" ...") 
        #reading csv dataset
        if self.dataset_name == 'fer2013': #change dataset***
            if self.path_csv is None:
                raw_data = pd.read_csv('../fer2013.csv')
            else:
                raw_data = pd.read_csv(self.path_csv)
        else:
            raise Exception("dataset not supported:"+str(self.dataset_name))

        #reading train and test split 
        n_train = np.sum(raw_data['Usage'] == 'Training')
        n_test = np.sum(raw_data['Usage'] != 'Training')
        assert n_train + n_test == len(raw_data)

        #"batch" of training and test data (#train/test samples, img_w, img_h, img_ch, dataType)
        self.img_vect_train = np.zeros( (n_train, self.img_res[0], self.img_res[1],
                                         self.img_res[2]), 'float32')
        self.img_vect_test = np.zeros( (n_test, self.img_res[0], self.img_res[1],
                                        self.img_res[2]), 'float32')
        self.lab_vect_train = np.zeros(n_train, 'int32')
        self.lab_vect_test = np.zeros(n_test, 'int32')

        i_train , i_test = 0,0
        #pass throught all data
        print("passing throught all data...")
        for i in range(len(raw_data)):
            
            #get pixels for i data
            img = raw_data["pixels"][i] 
            x_pixels = np.array(img.split(" "), 'float32')
            #normalize
            if self.normalize:
                x_pixels = x_pixels/127.5 - 1.
            #reshape into image matrix
            x_pixels = x_pixels.reshape(self.img_res)
            #get set (train or test)
            us = raw_data["Usage"][i]
            #save into image vect set for training or test
            if us == 'Training':            
                self.img_vect_train[i_train] = x_pixels
                self.lab_vect_train[i_train] = int(raw_data["emotion"][i]) 
                i_train = i_train + 1
            else:
                self.img_vect_test[i_test] = x_pixels
                self.lab_vect_test[i_test] = int(raw_data["emotion"][i]) 
                i_test = i_test + 1

        #for check 
        assert i_train == len(self.img_vect_train) 
        assert i_train == len(self.lab_vect_train) 
        assert i_test == len(self.lab_vect_test) 
        assert i_test == len(self.img_vect_test) 

        print("> loaded train:",len(self.img_vect_train),"   - test:",len(self.lab_vect_test) )

        #make RGB test/train images "batch" (#samples, width, high, 3)
        print("converting img to RGB...")
        self.img_vect_test_RGB = np.zeros((self.img_vect_test.shape[0], self.img_res[0],
                                            self.img_res[1], 3))
        
        #go through test RGB and convert gray-->RGB
        for i in range(self.img_vect_test_RGB.shape[0]):
            self.img_vect_test_RGB[i] = cv2.cvtColor(self.img_vect_test[i], cv2.COLOR_GRAY2RGB)

        print("ready test RGB!")

        self.img_vect_train_RGB = np.zeros((self.img_vect_train.shape[0],self.img_res[0],self.img_res[1],3))
        for i in range(self.img_vect_train_RGB.shape[0]):
            self.img_vect_train_RGB[i] = cv2.cvtColor(self.img_vect_train[i], cv2.COLOR_GRAY2RGB)
        print("ready train RGB!")

        ## its seems to we can read the images at the same time (changed***)
        #leo = cv2.imread('./images/leo_gray__crop_48_48.jpg', cv2.IMREAD_GRAYSCALE )
        ##batch de uno
        #self.leo = leo.reshape((1, self.img_res[0], self.img_res[1], self.img_res[2]))
        #self.leo_lab = 6 * np.ones(1, 'int32' ) # 6 for neutral 

        #when we use test data
        print("info de use_test_in_batch: ", self.use_test_in_batch)
        if self.use_test_in_batch:
            #revisar por que no esta el metodo leo_lab
            self.lab_vect_train = np.concatenate([self.lab_vect_train, self.lab_vect_test, self.leo_lab])
            self.img_vect_train = np.concatenate([self.img_vect_train, self.img_vect_test, self.leo])

    def load_leo(self):
        """Return label and image from reading
        """
        return self.leo_lab , self.leo

    def load_data(self, domain=None, batch_size=1, is_testing=False, convertRGB=False):
        """Load data function: load batch of data

        Parameters
        ------------
        domain: int- class label 
        batch_size: int- 
        is_testing: boolean- test or not
        convertRGB: boolean- to make RGB images

        Return
        ------------
        labels and images batch
        """
        if is_testing:
            #when label class was not given
            if domain is None:
                idx = np.random.choice(self.img_vect_test.shape[0], size=batch_size)
            else:                
                assert domain in [0,1,2,3,4,5,6]# for check that label given is correct
                idx0 = np.argwhere(self.lab_vect_test == domain)#get shape of data with label to work 
                idx1 = np.random.choice(idx0.shape[0], size=batch_size)#random choice
                idx = idx0[idx1]#from general data with the label we get the random data selected
                idx = np.squeeze(idx)#check size dimensions of idx***
            batch_images = self.img_vect_test[idx]
            labels = self.lab_vect_test[idx]
            #same for train data
        else:
            if domain is None:
                idx = np.random.choice(self.lab_vect_train.shape[0],size=batch_size)
            else:                
                assert domain in [0,1,2,3,4,5,6]
                idx0 = np.argwhere(self.lab_vect_train == domain) 
                idx1 = np.random.choice(idx0.shape[0],size=batch_size)
                idx = idx0[idx1]
                idx = np.squeeze(idx)
            batch_images = self.img_vect_train[idx]
            labels = self.lab_vect_train[idx]

        batch_images = np.resize(batch_images, (batch_size, self.img_res[0], self.img_res[1],
                                self.img_res[2]))

        if convertRGB:            
            _batch_images = np.zeros((batch_size, self.img_res[0], self.img_res[1], 3))
            for i in range(batch_size):
                _batch_images[i] = cv2.cvtColor(batch_images[i], cv2.COLOR_GRAY2RGB)
            batch_images = _batch_images

        if is_testing:
            return labels , batch_images
        for i in range(batch_size):
            if np.random.random() > 0.5:#check its meaning***
                batch_images[i] = np.fliplr(batch_images[i]) #for column flip (its needed?***)
        return labels , batch_images

    def load_batch(self, domain=None, batch_size=1, is_testing=False , convertRGB=False):
        """
        Parameters:
        --------------
        domain: int- label class
        batch_size: int- amount of images to be treated
        is_testing: boolean- for testing pourposes
        convertRGB: boolean- for get RGB images

        Return:
        --------------
        labels and their respective batch images
        """
        if is_testing:
            raise Exception("not supported")
        self.n_batches = int(len(self.img_vect_train) / batch_size)
        total_samples = self.n_batches * batch_size
        for i in range(self.n_batches):                       
            if domain is None:
                idx = np.random.choice(self.lab_vect_train.shape[0], size=batch_size)
            else:               
                assert domain in list(range(7))
                idx0 = np.argwhere(self.lab_vect_train == domain) 
                idx1 = np.random.choice(idx0.shape[0], size=batch_size)
                idx = idx0[idx1]
                idx = np.squeeze(idx)
            batch_images = self.img_vect_train[idx]
            labels = self.lab_vect_train[idx]
            for i in range(batch_size):
                if np.random.random() > 0.5:#check its meaning***
                    batch_images[i] = np.fliplr(batch_images[i]) #for column flip (its needed?***)
            batch_images = np.resize(batch_images, (batch_size,self.img_res[0],self.img_res[1],self.img_res[2]))
            if convertRGB:
                _batch_images = np.zeros((batch_size, self.img_res[0], self.img_res[1],3))
                for i in range(batch_size):
                    _batch_images[i] = cv2.cvtColor(batch_images[i], cv2.COLOR_GRAY2RGB)
                batch_images = _batch_images
            yield labels , batch_images

            #Nota: no entiendo muy bien la diferencia entre los metodos load_batch y load_data***


    def load_batch_AB(self, domain=None, batch_size=1, is_testing=False):
        """Load batch of data from two domains (A and B)
        Parameters:
        ---------------
        domain: array- labels class
        batch_size: int- amount of data to be loaded
        is_testing: boolean- for testing pourposes

        Return:
        ---------------
        Batch images from domains A and B
        Respective labels for data from both domains
        """
        if is_testing:#it seems to no support testing (make is_testing=False always?***)
            raise Exception("not supported")
        self.n_batches = int(len(self.img_vect_train) / batch_size)
        total_samples = self.n_batches * batch_size
        for i in range(self.n_batches):            
            assert domain is not None #check if domain is not empty 
            assert type(domain) is list #domain type must be list format
            #check both domains belong to labels between [0,6] 
            assert domain[0] in list(range(7))
            assert domain[1] in list(range(7))
            assert domain[0] != domain[1]#check different domains
            domain_A , domain_B = domain[0] , domain[1]
            # domain_A
            idx0 = np.argwhere(self.lab_vect_train == domain_A) 
            idx1 = np.random.choice(idx0.shape[0],size=batch_size)
            idx = idx0[idx1]
            idx = np.squeeze(idx)
            batch_images_A = self.img_vect_train[idx]
            labels_A = self.lab_vect_train[idx]
            for i in range(batch_size):
                if np.random.random() > 10.5:#check its meaning***
                    batch_images_A[i] = np.fliplr(batch_images_A[i])#for column flip (its needed?***)
            batch_images_A = np.resize(batch_images_A, (batch_size,self.img_res[0],self.img_res[1],self.img_res[2]))
            # domain_B
            idx0 = np.argwhere(self.lab_vect_train == domain_B) 
            idx1 = np.random.choice(idx0.shape[0],size=batch_size)
            idx = idx0[idx1]
            idx = np.squeeze(idx)
            batch_images_B = self.img_vect_train[idx]
            labels_B = self.lab_vect_train[idx]
            for i in range(batch_size):
                if np.random.random() > 10.5:#check its meaning***
                    batch_images_B[i] = np.fliplr(batch_images_B[i])#for column flip (its needed?***)
            batch_images_B = np.resize(batch_images_B, (batch_size,self.img_res[0],self.img_res[1],self.img_res[2]))

            yield labels_A , batch_images_A , labels_B , batch_images_B

# <font color='red'>**Models**</font>
## **Useful libraries**

In [5]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-1cbd6tls
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=30a55e2680a88bceb66343ece3abf2034bf3962c6ed3eda6ef610496379a6618
  Stored in directory: /tmp/pip-ephem-wheel-cache-0098c_kg/wheels/67/d2/f4/96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
Successfully built keras-contrib
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import Reshape
import datetime
import sys
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
import numpy as np
import os
import random 
from keras.layers import Conv2DTranspose, BatchNormalization
import tensorflow as tf 

from tensorflow.keras.utils import to_categorical

### Useful methods

In [7]:
def get_dim_conv(dim,f,p,s):
    """Function to calculate output conv shape
    Parameters:
    -----------
    dim: 
    f: int- amount of filters
    p: int- padding
    s: int- stride 

    Return:
    -----------
    new output dimension
    """
    return int((dim+2*p-f)/2+1)

In [8]:
def build_generator_enc_dec(img_shape, gf, num_classes, channels, num_layers=4, f_size=4, 
                            tranform_layer=False):
    """U-Net Generator
    Parameters:
    ------------
    img_shape: array- image shape
    gf: int- amount of filters check***
    num_classes: int- classes to be taken into account
    num_layers: int- number of layers in net
    f_size: int- filter size
    transform_layer: boolean- 

    Return: enconder and decoder nets
    """
    
    def conv2d(layer_input, filters, f_size=f_size):
        """Layers used during downsampling"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='valid')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        d = InstanceNormalization()(d)
        return d    

    #no se usa
    #def __deconv2d(layer_input, skip_input, filters, f_size=f_size, dropout_rate=0):
    #    """Layers used during upsampling"""
    #    u = UpSampling2D(size=2)(layer_input)
    #    u = Conv2D(filters, kernel_size=f_size, strides=1, padding='valid', activation='relu')(u)
    #    if dropout_rate:
    #        u = Dropout(dropout_rate)(u)
    #    u = InstanceNormalization()(u)
    #    u = Concatenate()([u, skip_input])
    #    return u

    def deconv2d(layer_input, skip_input, filters, f_size=f_size, dropout_rate=0, output_padding=None):
        """Layers used during upsampling"""
        u = Conv2DTranspose(filters=filters, kernel_size=f_size, strides=2, activation='relu',
                            output_padding=output_padding)(layer_input)
        if dropout_rate:
          u = Dropout(dropout_rate)(u)
        u = InstanceNormalization()(u)
        u = Concatenate()([u, skip_input])
        return u

    # Image input layer
    img = Input(shape=img_shape)

    # Downsampling
    d = img 
    zs = [] 
    dims = []
    _dim = img_shape[0]
    for i in range(num_layers):
        d = conv2d(d, gf*2**i)# add by 2 as we go deeper in the net
        zs.append(d)
        _dim = get_dim_conv(_dim,f_size,0,2)
        dims.append((_dim,gf*2**i))
        print("D:",_dim,gf*2**i)
    G_enc = Model(img,zs)#encoder net

    #### 
    # = Input(shape=(24, 24, 32))
    #d2_ = Input(shape=(12, 12, 64))
    #d3_ = Input(shape=(6, 6, 128))
    #d4_ = Input(shape=(3, 3, 256))

    _zs = [] 
    d_ , c_ = dims.pop()#check dims dim***
    print(0,d_,c_)
    i_ = Input(shape=(d_, d_, c_))
    _zs.append(i_)
    label = Input(shape=(num_classes,), dtype='float32')
    label_r = Reshape((1,1,num_classes))(label)

    u = concatenate([i_, label_r],axis=-1)

    ## transf (why?***)
    if tranform_layer:
        tr = Flatten()(u)
        tr = Dense(c_+num_classes)(tr)
        tr = LeakyReLU(alpha=0.2)(tr)
        u = Reshape((1,1,c_+num_classes))(tr)
    ##

    u = Conv2D(c_, kernel_size=1, strides=1, padding='valid')(u) ## 1x1 conv 

    # Upsampling
    for i in range(num_layers-1):
        _ch = gf*2**((num_layers-2)-i)
        d_ , c_ = dims.pop()
        print(i,d_,c_)
        i_ = Input(shape=(d_, d_, c_))
        _zs.append(i_)
        if i == 2:
            u = deconv2d(u, i_, _ch, output_padding=1)
        else:
            u = deconv2d(u, i_, _ch)

    #u4 = UpSampling2D(size=2)(u)
    #output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    u = Conv2DTranspose(filters=channels, kernel_size=f_size, strides=2, activation='tanh', output_padding=None)(u)


    _zs.reverse()
    _zs.append(label)
    G_dec = Model(_zs,u) #decoder net

    return G_enc , G_dec

In [9]:
def build_discriminator(img_shape, df, num_classes, num_layers=4, act_multi_label='softmax'):
    """Build discriminator function: net for discriminate real from fake data
    Parameters:
    -----------
    img_shape: array- (w,h,c)
    df: int- dimension filters check***
    num_layers: int- amount of model's layers
    act_multi_label: string- activation function

    Return: discriminator model
    """

    def d_layer(layer_input, filters, f_size=4, normalization=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='valid')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            d = InstanceNormalization()(d)
        return d

    img = Input(shape=img_shape)

    #label = Input(shape=(1,), dtype='int32')
    #label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
    #flat_img = Flatten()(img)
    #model_input = multiply([flat_img, label_embedding])
    #d0 = Reshape(self.img_shape)(model_input)

    d = img 
    for i in range(num_layers):
        #normalize all layers except the 1st one
        _norm = False if i == 0 else True 
        d = d_layer(d, df*2**i, normalization=_norm)

    flat_repr = Flatten()(d)#flat representation of the last layer

    #validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    print("flat_repr.get_shape().as_list():",flat_repr.get_shape().as_list())
    print("flat_repr.get_shape().as_list()[1:]:",flat_repr.get_shape().as_list()[1:])

    #Dense neural net
    #Part to address the real or fake discrimination
    gan_logit = Dense(df*2**(num_layers-1))(flat_repr)
    gan_logit = LeakyReLU(alpha=0.2)(gan_logit)
    gan_prob = Dense(1, activation='sigmoid')(gan_logit)

    #Part to address the class classification
    class_logit = Dense(df*2**(num_layers-1))(flat_repr)
    class_logit = LeakyReLU(alpha=0.2)(class_logit)
    class_prob = Dense(num_classes, activation=act_multi_label)(class_logit)

    #### 
    #label = Input(shape=(1,), dtype='int32')
    #label_embedding = Flatten()(Embedding(self.num_classes, 9)(label))
    #flat_img = Flatten()(validity)
    #d44 = multiply([flat_img, label_embedding])
    #d444 = Reshape(validity.get_shape().as_list()[1:])(d44)
    ####

    return Model(img, [gan_prob, class_prob])

# <font color='red'>**Conditional cycleGan network**</font>
## **Useful libraries**

In [10]:
from __future__ import print_function, division
import scipy

from tensorflow.keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.models import Sequential, Model
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import Reshape
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
import numpy as np
import pandas as pd 
import os
import random 

import tensorflow as tf 

from tensorflow.keras.utils import to_categorical
import argparse
from sklearn.metrics import accuracy_score

#from  models import *

In [11]:
class CCycleGAN():
    """ Conditional cycleGan: model initialization (generator and discriminator nets) and training,
    receive image shape,
    amount of classes to be taken into account, 
    weight losses for generator and discriminator nets,
    load the dataset.

    Parameters
    ------------
    img_rows and img_cols: int- rows and cols for image to work with
    channels: int- amount of image channels
    num_classes: int- amount of classes to be taken into account
    d_gan_loss_w: int- discriminator loss weight
    d_cl_loss_w: int- discriminator loss weight for class tag
    g_gan_loss_w: int- generator loss weight
    g_cl_loss_w: int- generator loss weight for class tag
    ---> rec_loss_w: int- cycle consistency loss weight (check)
    adam_lr: float- learning rate
    adam_beta_1: float- parameters for adam rule
    adam_beta_2: float- parameters for adam rule 
    """

    #values assignment
    def __init__(self,img_rows = 48,img_cols = 48,channels = 1, num_classes=7, d_gan_loss_w=1,
      d_cl_loss_w=1, g_gan_loss_w=1, g_cl_loss_w=1, rec_loss_w=1, adam_lr=0.0002, adam_beta_1=0.5,
      adam_beta_2=0.999):
        
        # Input shape
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = num_classes

        # Loss weights 
        self.d_gan_loss_w = d_gan_loss_w
        self.d_cl_loss_w = d_cl_loss_w
        self.g_gan_loss_w = g_gan_loss_w
        self.g_cl_loss_w = g_cl_loss_w
        self.rec_loss_w = rec_loss_w

        # optmizer params 
        self.adam_lr = adam_lr
        self.adam_beta_1 = adam_beta_1
        self.adam_beta_2 = adam_beta_2

        # Configure data loader
        self.dataset_name = 'fer2013' #maybe changed dataset name (ver importancia del nombre?***)
        # TALE SECOND PART (pasar a file: data_loader.py)
        self.data_loader = DataLoader(dataset_name=self.dataset_name,img_res=self.img_shape,
                                      use_test_in_batch=False)
        # label dict
        self.lab_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise",
                         6: "Neutral"}

        # Number of filters in the first layer of Generator and Discriminator
        self.gf = 32
        self.df = 64

        optimizer = Adam(self.adam_lr, self.adam_beta_1, self.adam_beta_2) 

        # Build and compile the discriminators (models.py method)
        self.d = build_discriminator(img_shape=self.img_shape, df=64, num_classes=self.num_classes,
                                    act_multi_label='sigmoid')
        print("******** Discriminator/Classifier ********")
        self.d.summary()
        self.d.compile(loss=[
            'binary_crossentropy',  # gan
            'binary_crossentropy'   # class 
            ],
            optimizer=optimizer,
            metrics=['accuracy'],
            loss_weights=[
            self.d_gan_loss_w , # gan
            self.d_cl_loss_w   # class
            ])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators (here i go#1)
        self.g_enc , self.g_dec = build_generator_enc_dec(img_shape=(48,48,1), gf=64, num_classes=7,
                                                          channels=1, tranform_layer=True)
        print("******** Generator_ENC ********")
        self.g_enc.summary()
        print("******** Generator_DEC ********")
        self.g_dec.summary()

        # Input images from both domains
        img = Input(shape=self.img_shape)
        label0 = Input(shape=(self.num_classes,))
        label1 = Input(shape=(self.num_classes,))

        # Translate images to the other domain
        z1,z2,z3,z4 = self.g_enc(img)
        fake = self.g_dec([z1,z2,z3,z4,label1])

        # Translate images back to original domain
        reconstr = self.g_dec([z1,z2,z3,z4,label0])

        # For the combined model we will only train the generators
        self.d.trainable = False

        # Discriminators determines validity of translated images gan_prob,
        # class_prob [label,img], [gan_prob,class_prob]
        gan_valid , class_valid = self.d(fake)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img,label0,label1], outputs=[ gan_valid, class_valid, reconstr])
        self.combined.compile(loss=['binary_crossentropy','categorical_crossentropy',
                                    'mae'],
                              loss_weights=[                                            
                                            self.g_gan_loss_w, # g_loss gan 
                                            self.g_cl_loss_w, # g_loss class  
                                            self.rec_loss_w # reconstruction loss
                                          ],
                            optimizer=optimizer)

        print("******** Combined model ********")
        self.combined.summary()

    def generate_new_labels(self,labels0):
        labels1 = [] 
        for i in range(len(labels0)):
            allowed_values = list(range(0, self.num_classes))
            allowed_values.remove(labels0[i])
            labels1.append(random.choice(allowed_values))
        return np.array(labels1,'int32')

    def generate_new_labels_all(self, labels0):
        #called from training procedure check***
        """Function for keep label values different from original labels
        Parameter:
        labels0: array- real label class list
        Return: array with all labels different from original label class
        """
        labels_all = [] 
        for i in range(len(labels0)):
            allowed_values = list(range(0, self.num_classes))
            allowed_values.remove(labels0[i])
            labels_all.append(np.array(allowed_values,'int32'))
        return np.array(labels_all,'int32')

    def train(self, epochs, batch_size=1, sample_interval=50 , d_g_ratio=5):
        """Conditional cycleGan training function
        Parameters:
        ------------
        epochs: int- amount of epochs to train model
        batch_size: int- number of samples to be taken into account for each update step
        sample_interval: int- check***
        d_g_ratio: int- epoch frequency for decay learning rate check***
        """

        start_time = datetime.datetime.now()
        # logs 
        epoch_history, batch_i_history,  = [] , []   
        d_gan_loss_history, d_gan_accuracy_history, d_cl_loss_history, d_cl_accuracy_history = [], [], [], [] 
        g_gan_loss_history, g_cl_loss_history = [] , [] 
        reconstr_history = [] 

        # Adversarial loss ground truths
        valid = np.ones((batch_size,1) )
        fake = np.zeros((batch_size,1) )

        null_labels = np.zeros((batch_size,7) )

        for epoch in range(epochs):
            for batch_i, (labels0 , imgs) in enumerate(self.data_loader.load_batch(batch_size=batch_size)):
                labels1_all = self.generate_new_labels_all(labels0)
                labels0_cat = to_categorical(labels0, num_classes=self.num_classes)
                #
                labels1_all_1 = to_categorical(labels1_all[:,0], num_classes=self.num_classes)
                labels1_all_2 = to_categorical(labels1_all[:,1], num_classes=self.num_classes)
                labels1_all_3 = to_categorical(labels1_all[:,2], num_classes=self.num_classes)
                labels1_all_4 = to_categorical(labels1_all[:,3], num_classes=self.num_classes)
                labels1_all_5 = to_categorical(labels1_all[:,4], num_classes=self.num_classes)
                labels1_all_6 = to_categorical(labels1_all[:,5], num_classes=self.num_classes)

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                zs1,zs2,zs3,zs4 = self.g_enc.predict(imgs)#check what encoder returns***
                fakes_1 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_1])
                fakes_2 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_2])
                fakes_3 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_3])
                fakes_4 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_4])
                fakes_5 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_5])
                fakes_6 = self.g_dec.predict([zs1,zs2,zs3,zs4,labels1_all_6])

                # Train the discriminators (original images = real / translated = Fake)
                idx = np.random.permutation(self.num_classes*labels0.shape[0])
                _labels_cat = np.concatenate([labels0_cat,
                                              null_labels,
                                              null_labels,
                                              null_labels,
                                              null_labels,
                                              null_labels,
                                              null_labels])
                _imgs = np.concatenate([imgs,
                                        fakes_1,
                                        fakes_2,
                                        fakes_3,
                                        fakes_4,
                                        fakes_5,
                                        fakes_6])
                _vf = np.concatenate([valid, fake, fake, fake, fake, fake, fake])
                _labels_cat = _labels_cat[idx]
                _imgs = _imgs[idx]
                _vf = _vf[idx]

                d_loss  = self.d.train_on_batch(_imgs, [_vf,_labels_cat])

                if batch_i % d_g_ratio == 0:
                    # ------------------
                    #  Train Generators
                    # ------------------
                    _imgs = np.concatenate([imgs,                                                     
                                          imgs,
                                          imgs,
                                          imgs,
                                          imgs,
                                          imgs])

                    _labels0_cat = np.concatenate([labels0_cat,                                                               
                                                labels0_cat,
                                                labels0_cat,
                                                labels0_cat,
                                                labels0_cat,
                                                labels0_cat])

                    _labels1_all_other = np.concatenate([labels1_all_1,                                                                                
                                                      labels1_all_2,
                                                      labels1_all_3,
                                                      labels1_all_4,
                                                      labels1_all_5,
                                                      labels1_all_6])

                    # I know this should be outside the loop;
                    # left here to make code more understandable 
                    _valid = np.concatenate([valid,                                                      
                                          valid,
                                          valid,
                                          valid,
                                          valid,
                                          valid])

                    idx = np.random.permutation((self.num_classes-1)*labels0.shape[0])
                    _imgs = _imgs[idx]
                    _labels0_cat = _labels0_cat[idx]
                    _labels1_all_other = _labels1_all_other[idx]
                    _valid = _valid[idx]

                    # Train the generators
                    g_loss = self.combined.train_on_batch([_imgs, _labels0_cat, _labels1_all_other],
                                                          [_valid, _labels1_all_other, _imgs])

                    elapsed_time = datetime.datetime.now() - start_time

                    print ("[Epoch %d/%d] [Batch %d/%d] [D_gan loss: %f, acc_gan: %3d%%] [D_cl loss: %f, acc_cl: %3d%%] [G_gan loss: %05f, G_cl: %05f, recon: %05f] time: %s " \
                      % ( epoch, epochs,
                          batch_i, self.data_loader.n_batches,
                          d_loss[1],100*d_loss[3],d_loss[2],100*d_loss[4],
                          g_loss[1],g_loss[2],g_loss[3],
                          elapsed_time))

                    # log
                    epoch_history.append(epoch) 
                    batch_i_history.append(batch_i)
                    d_gan_loss_history.append(d_loss[1])
                    d_gan_accuracy_history.append(100*d_loss[3])
                    d_cl_loss_history.append(d_loss[2])
                    d_cl_accuracy_history.append(100*d_loss[4])
                    g_gan_loss_history.append(g_loss[1])
                    g_cl_loss_history.append(g_loss[2])
                    reconstr_history.append(g_loss[3])

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:    
                    self.sample_images(epoch, batch_i)
                    #self.sample_images(epoch, batch_i,use_leo=True)

                    train_history = pd.DataFrame({
                      'epoch': epoch_history, 
                      'batch': batch_i_history, 
                      'd_gan_loss': d_gan_loss_history, 
                      'd_gan_accuracy' : d_gan_accuracy_history,
                      'd_cl_loss': d_cl_loss_history, 
                      'd_cl_accuracy': d_cl_accuracy_history, 
                      'g_gan_loss': g_gan_loss_history, 
                      'g_cl_loss': g_cl_loss_history, 
                      'reconstr_loss': reconstr_history
                    })
                    train_history.to_csv(str(sys.argv[0]).split('.')[0]+'_train_log.csv',index=False)

    def sample_images(self, epoch, batch_i, use_leo=False):
        """Function to save a batch test samples
        Parameters:
        ------------
        epoch: int- epoch where the interval save is done
        batch_i: int- number of batch where we want to save
        """
        ## disc
        labels0_d , imgs_d = self.data_loader.load_data(batch_size=64, is_testing=True)
        #predicting images with discriminator net
        gan_pred_prob, class_pred_prob = self.d.predict(imgs_d)

        gan_pred = (gan_pred_prob > 0.5)*1.0
        gan_pred = gan_pred.reshape((64,))

        class_pred = np.argmax(class_pred_prob,axis=1)

        gan_test_accuracy = accuracy_score(y_true=np.ones(64), y_pred=gan_pred)
        class_test_accuracy = accuracy_score(y_true=labels0_d, y_pred=class_pred)

        print("*** TEST *** [D_gan accuracy :",gan_test_accuracy,"] [D_cl accuracy :", class_test_accuracy,"]")

        ## gen         
        if use_leo:
            labels0_ , imgs_ = self.data_loader.load_leo()#load_leo() why?***
        else:
            labels0_ , imgs_ = self.data_loader.load_data(batch_size=1, is_testing=True)
        labels1_all = self.generate_new_labels_all(labels0_)

        labels0_cat = to_categorical(labels0_, num_classes=self.num_classes)
        labels1_all_1 = to_categorical(labels1_all[:,0], num_classes=self.num_classes)
        labels1_all_2 = to_categorical(labels1_all[:,1], num_classes=self.num_classes)
        labels1_all_3 = to_categorical(labels1_all[:,2], num_classes=self.num_classes)
        labels1_all_4 = to_categorical(labels1_all[:,3], num_classes=self.num_classes)
        labels1_all_5 = to_categorical(labels1_all[:,4], num_classes=self.num_classes)
        labels1_all_6 = to_categorical(labels1_all[:,5], num_classes=self.num_classes)

        # Translate images 
        zs1_,zs2_,zs3_,zs4_ = self.g_enc.predict(imgs_)
        fake_1 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_1])
        fake_2 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_2])
        fake_3 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_3])
        fake_4 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_4])
        fake_5 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_5])
        fake_6 = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels1_all_6])

        # Reconstruct image 
        reconstr_ = self.g_dec.predict([zs1_,zs2_,zs3_,zs4_,labels0_cat])

        gen_imgs = np.concatenate([imgs_,                              
                                  fake_1, 
                                  fake_2, 
                                  fake_3, 
                                  fake_4, 
                                  fake_5, 
                                  fake_6,
                                  reconstr_])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5# check the rol of 0.5***

        titles = ['Orig:'+str(self.lab_dict[labels0_.item(0)]), 
                  'Trans:'+str(self.lab_dict[labels1_all[:,0].item(0)]),
                  'Trans:'+str(self.lab_dict[labels1_all[:,1].item(0)]),
                  'Trans:'+str(self.lab_dict[labels1_all[:,2].item(0)]),
                  'Trans:'+str(self.lab_dict[labels1_all[:,3].item(0)]),
                  'Trans:'+str(self.lab_dict[labels1_all[:,4].item(0)]),
                  'Trans:'+str(self.lab_dict[labels1_all[:,5].item(0)]),
                  'Reconstr.']
        r, c = 2, 4#for rows and cols
        fig, axs = plt.subplots(r, c)

        plt.subplots_adjust(hspace=0)

        if not os.path.exists( "images/%s/"% (self.dataset_name)):
            os.makedirs( "images/%s/"% (self.dataset_name)  )

        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt].reshape((self.img_rows,self.img_cols)),cmap='gray')
                axs[i,j].set_title(titles[cnt])
                axs[i,j].axis('off')
                cnt += 1

        if use_leo:
            fig.savefig("images/%s/%d_%d_leo.png" % (self.dataset_name, epoch, batch_i))
        else:
            fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

**Second idea** falta identar

In [ ]:
class CCycleGAN():

  def __init__(self, img_rows=48, img_cols=48, channels=1, num_classes=7, latent_dim=99, 
               PREFIX='saved_model/'):
    # Input shape
    self.img_rows = img_rows
    self.img_cols = img_cols
    self.channels = channels
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.num_classes = num_classes
    self.latent_dim = latent_dim
    self.PREFIX=PREFIX
    
    ## dict
    self.lab_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

    # Configure data loader
    self.dataset_name = 'fer2013'
    self.data_loader = DataLoader(dataset_name=self.dataset_name,img_res=self.img_shape)

    optimizer = Adam(0.0002, 0.5)

    # Build and compile the discriminators
    self.d = self.build_discriminator2()
    print("******** Discriminator ********")
    self.d.summary()
    self.d.compile(loss='categorical_crossentropy',
                   optimizer=optimizer,
                   metrics=['accuracy'])
  
  def build_discriminator2(self):    
    base_model  = ResNet50(weights= 'imagenet', include_top=False, input_shape= (48,48,3))
    
    # add a global spatial average pooling layer
    x = base_model.output
    latent_repr = GlobalAveragePooling2D()(x)
    
    # let's add a fully-connected layer
    f = Dense(1024, activation='relu')(latent_repr)
    predictions = Dense(self.num_classes, activation='softmax')(f)
    
    return Model(base_model.input, predictions)

  def train(self, epochs, batch_size=1, sample_interval=50):
    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    earlystopper = EarlyStopping(patience=20, verbose=1,monitor='val_acc',mode='max')
    checkpointer = ModelCheckpoint(self.PREFIX+'classifier_2.h5', verbose=1, save_best_only=True,monitor='val_acc',mode='max')
    reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=0.00001, verbose=1,monitor='val_acc',mode='max')
    results = self.d.fit(self.data_loader.img_vect_train_RGB, 
                        np_utils.to_categorical(self.data_loader.lab_vect_train,num_classes=self.num_classes),
                validation_data=[self.data_loader.img_vect_test_RGB,
                                  np_utils.to_categorical(self.data_loader.lab_vect_test,num_classes=self.num_classes)],
                batch_size=batch_size, epochs=epochs,
                callbacks=[earlystopper, checkpointer,reduce_lr], shuffle=True)

# <font color='red'>Main</font>

In [12]:
d_gan_loss_w =1
d_cl_loss_w =1
g_gan_loss_w =2
g_cl_loss_w =2
rec_loss_w =1
adam_lr =0.0002
adam_beta_1 =0.5
adam_beta_2 =0.999
epochs =170
batch_size =64
sample_interval =200

In [13]:
# CCycleGAN: THE TALE START HERE
gan = CCycleGAN(d_gan_loss_w=d_gan_loss_w, d_cl_loss_w=d_cl_loss_w,            
                g_gan_loss_w=g_gan_loss_w, g_cl_loss_w=g_cl_loss_w,
                rec_loss_w=rec_loss_w, adam_lr=adam_lr,
                adam_beta_1=adam_beta_1, adam_beta_2=adam_beta_2)

>> loading fer2013 ...
passing throught all data...
> loaded train: 28709    - test: 7178
converting img to RGB...
ready test RGB!
ready train RGB!
info de use_test_in_batch:  False
flat_repr.get_shape().as_list(): [None, 512]
flat_repr.get_shape().as_list()[1:]: [512]
******** Discriminator/Classifier ********
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 23, 23, 64)   1088        ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 23, 23, 64)   0           ['conv2d[0][0

******** Combined model ********
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 model_1 (Functional)           [(None, 23, 23, 64)  2754504     ['input_8[0][0]']                
                                , (None, 10, 10, 12                                               
                                8),                                                               
                                 (None, 4, 4, 256),                                               
                                 (None, 1, 1, 512)]                                               
                                                           

In [ ]:
gan.train(epochs=epochs, batch_size=batch_size, sample_interval=sample_interval)

[Epoch 0/170] [Batch 0/448] [D_gan loss: 0.907626, acc_gan:  41%] [D_cl loss: 0.783718, acc_cl:   2%] [G_gan loss: 7.246975, G_cl: 2.294716, recon: 0.518738] time: 0:01:05.744795 
*** TEST *** [D_gan accuracy : 0.0 ] [D_cl accuracy : 0.140625 ]
[Epoch 0/170] [Batch 5/448] [D_gan loss: 0.011208, acc_gan:  99%] [D_cl loss: 0.075165, acc_cl:   4%] [G_gan loss: 6.290006, G_cl: 2.097187, recon: 0.474926] time: 0:01:12.221241 
[Epoch 0/170] [Batch 10/448] [D_gan loss: 0.219880, acc_gan:  89%] [D_cl loss: 0.083749, acc_cl:   4%] [G_gan loss: 3.371043, G_cl: 2.222383, recon: 0.434080] time: 0:01:15.560228 
[Epoch 0/170] [Batch 15/448] [D_gan loss: 0.111448, acc_gan:  96%] [D_cl loss: 0.077660, acc_cl:   7%] [G_gan loss: 3.064933, G_cl: 2.076971, recon: 0.378742] time: 0:01:18.997579 
[Epoch 0/170] [Batch 20/448] [D_gan loss: 0.387211, acc_gan:  86%] [D_cl loss: 0.095040, acc_cl:   9%] [G_gan loss: 2.302874, G_cl: 2.166189, recon: 0.360182] time: 0:01:22.707562 
[Epoch 0/170] [Batch 25/448] [D_

[Epoch 0/170] [Batch 225/448] [D_gan loss: 0.218463, acc_gan:  93%] [D_cl loss: 0.081751, acc_cl:  12%] [G_gan loss: 2.924950, G_cl: 2.378883, recon: 0.260526] time: 0:03:41.835512 
[Epoch 0/170] [Batch 230/448] [D_gan loss: 0.175894, acc_gan:  95%] [D_cl loss: 0.073308, acc_cl:   3%] [G_gan loss: 3.374819, G_cl: 2.277999, recon: 0.265600] time: 0:03:45.113013 
[Epoch 0/170] [Batch 235/448] [D_gan loss: 0.120883, acc_gan:  94%] [D_cl loss: 0.064619, acc_cl:  27%] [G_gan loss: 3.917976, G_cl: 2.177194, recon: 0.270800] time: 0:03:48.250598 
[Epoch 0/170] [Batch 240/448] [D_gan loss: 0.240500, acc_gan:  92%] [D_cl loss: 0.082246, acc_cl:   3%] [G_gan loss: 2.998001, G_cl: 2.344796, recon: 0.259873] time: 0:03:51.637550 
[Epoch 0/170] [Batch 245/448] [D_gan loss: 0.132242, acc_gan:  97%] [D_cl loss: 0.071431, acc_cl:   3%] [G_gan loss: 3.419551, G_cl: 2.233060, recon: 0.261948] time: 0:03:54.883159 
[Epoch 0/170] [Batch 250/448] [D_gan loss: 0.108968, acc_gan:  96%] [D_cl loss: 0.067753, 

[Epoch 1/170] [Batch 0/448] [D_gan loss: 0.154729, acc_gan:  95%] [D_cl loss: 0.070752, acc_cl:  19%] [G_gan loss: 3.177279, G_cl: 2.266039, recon: 0.237589] time: 0:06:12.817559 
*** TEST *** [D_gan accuracy : 0.6875 ] [D_cl accuracy : 0.375 ]
[Epoch 1/170] [Batch 5/448] [D_gan loss: 0.144255, acc_gan:  95%] [D_cl loss: 0.071421, acc_cl:   4%] [G_gan loss: 3.144072, G_cl: 2.305977, recon: 0.234834] time: 0:06:17.015663 
[Epoch 1/170] [Batch 10/448] [D_gan loss: 0.174118, acc_gan:  94%] [D_cl loss: 0.071049, acc_cl:  22%] [G_gan loss: 2.826204, G_cl: 2.386853, recon: 0.239038] time: 0:06:20.322493 
[Epoch 1/170] [Batch 15/448] [D_gan loss: 0.107558, acc_gan:  96%] [D_cl loss: 0.066903, acc_cl:   6%] [G_gan loss: 3.720191, G_cl: 2.315405, recon: 0.236433] time: 0:06:24.071752 
[Epoch 1/170] [Batch 20/448] [D_gan loss: 0.162209, acc_gan:  93%] [D_cl loss: 0.069542, acc_cl:   6%] [G_gan loss: 2.998634, G_cl: 2.377936, recon: 0.229469] time: 0:06:27.561130 
[Epoch 1/170] [Batch 25/448] [D_

[Epoch 1/170] [Batch 225/448] [D_gan loss: 0.138564, acc_gan:  95%] [D_cl loss: 0.066200, acc_cl:  10%] [G_gan loss: 3.298430, G_cl: 2.540844, recon: 0.236860] time: 0:08:47.475293 
[Epoch 1/170] [Batch 230/448] [D_gan loss: 0.077278, acc_gan:  97%] [D_cl loss: 0.060266, acc_cl:  21%] [G_gan loss: 4.087531, G_cl: 2.420224, recon: 0.209461] time: 0:08:50.845466 
[Epoch 1/170] [Batch 235/448] [D_gan loss: 0.300718, acc_gan:  90%] [D_cl loss: 0.086613, acc_cl:  46%] [G_gan loss: 2.493769, G_cl: 2.756335, recon: 0.215217] time: 0:08:54.189455 
[Epoch 1/170] [Batch 240/448] [D_gan loss: 0.142543, acc_gan:  93%] [D_cl loss: 0.067446, acc_cl:  15%] [G_gan loss: 3.619477, G_cl: 2.563437, recon: 0.220105] time: 0:08:57.556233 
[Epoch 1/170] [Batch 245/448] [D_gan loss: 0.134851, acc_gan:  94%] [D_cl loss: 0.067648, acc_cl:   6%] [G_gan loss: 3.725761, G_cl: 2.471151, recon: 0.224011] time: 0:09:00.864005 
[Epoch 1/170] [Batch 250/448] [D_gan loss: 0.134465, acc_gan:  94%] [D_cl loss: 0.060146, 

[Epoch 2/170] [Batch 0/448] [D_gan loss: 0.003327, acc_gan: 100%] [D_cl loss: 0.045371, acc_cl:   8%] [G_gan loss: 8.651537, G_cl: 2.113062, recon: 0.258415] time: 0:11:17.974443 
*** TEST *** [D_gan accuracy : 0.984375 ] [D_cl accuracy : 0.421875 ]
[Epoch 2/170] [Batch 5/448] [D_gan loss: 0.005389, acc_gan: 100%] [D_cl loss: 0.050482, acc_cl:   8%] [G_gan loss: 8.573068, G_cl: 2.102407, recon: 0.244008] time: 0:11:22.265177 
[Epoch 2/170] [Batch 10/448] [D_gan loss: 0.003061, acc_gan: 100%] [D_cl loss: 0.046888, acc_cl:   8%] [G_gan loss: 8.191614, G_cl: 2.099402, recon: 0.243728] time: 0:11:25.586405 
[Epoch 2/170] [Batch 15/448] [D_gan loss: 0.184651, acc_gan:  94%] [D_cl loss: 0.071439, acc_cl:   5%] [G_gan loss: 4.886298, G_cl: 2.253090, recon: 0.253493] time: 0:11:28.848315 
[Epoch 2/170] [Batch 20/448] [D_gan loss: 0.038681, acc_gan:  98%] [D_cl loss: 0.049754, acc_cl:  10%] [G_gan loss: 5.176938, G_cl: 2.107440, recon: 0.270426] time: 0:11:32.105561 
[Epoch 2/170] [Batch 25/448

...

[Epoch 16/170] [Batch 0/448] [D_gan loss: 0.203484, acc_gan:  91%] [D_cl loss: 0.062100, acc_cl:  21%] [G_gan loss: 3.782595, G_cl: 1.343394, recon: 0.162284] time: 1:23:07.127850 
*** TEST *** [D_gan accuracy : 0.78125 ] [D_cl accuracy : 0.53125 ]
[Epoch 16/170] [Batch 5/448] [D_gan loss: 0.191639, acc_gan:  93%] [D_cl loss: 0.064378, acc_cl:  34%] [G_gan loss: 3.612827, G_cl: 1.268014, recon: 0.160655] time: 1:23:12.375532 
[Epoch 16/170] [Batch 10/448] [D_gan loss: 0.136315, acc_gan:  96%] [D_cl loss: 0.058995, acc_cl:  26%] [G_gan loss: 3.437132, G_cl: 1.408611, recon: 0.166825] time: 1:23:15.892493 
[Epoch 16/170] [Batch 15/448] [D_gan loss: 0.082800, acc_gan:  97%] [D_cl loss: 0.048876, acc_cl:  24%] [G_gan loss: 4.150486, G_cl: 1.557819, recon: 0.165053] time: 1:23:19.372629 
[Epoch 16/170] [Batch 20/448] [D_gan loss: 0.047669, acc_gan:  99%] [D_cl loss: 0.043258, acc_cl:  21%] [G_gan loss: 4.529045, G_cl: 1.414264, recon: 0.166051] time: 1:23:22.721852 
[Epoch 16/170] [Batch 25

[Epoch 16/170] [Batch 225/448] [D_gan loss: 0.068035, acc_gan:  97%] [D_cl loss: 0.049867, acc_cl:  16%] [G_gan loss: 4.773312, G_cl: 1.306900, recon: 0.192841] time: 1:25:43.823722 
[Epoch 16/170] [Batch 230/448] [D_gan loss: 0.115525, acc_gan:  97%] [D_cl loss: 0.054452, acc_cl:  19%] [G_gan loss: 4.567724, G_cl: 1.115642, recon: 0.201101] time: 1:25:47.099373 
[Epoch 16/170] [Batch 235/448] [D_gan loss: 0.103626, acc_gan:  95%] [D_cl loss: 0.052842, acc_cl:  23%] [G_gan loss: 4.510727, G_cl: 1.067113, recon: 0.200176] time: 1:25:50.513115 
[Epoch 16/170] [Batch 240/448] [D_gan loss: 0.115327, acc_gan:  96%] [D_cl loss: 0.053830, acc_cl:  28%] [G_gan loss: 4.026173, G_cl: 1.109033, recon: 0.184243] time: 1:25:54.093776 
[Epoch 16/170] [Batch 245/448] [D_gan loss: 0.100115, acc_gan:  96%] [D_cl loss: 0.050886, acc_cl:  26%] [G_gan loss: 5.146345, G_cl: 1.181902, recon: 0.190479] time: 1:25:57.327384 
[Epoch 16/170] [Batch 250/448] [D_gan loss: 0.081138, acc_gan:  97%] [D_cl loss: 0.04

[Epoch 17/170] [Batch 0/448] [D_gan loss: 0.211697, acc_gan:  91%] [D_cl loss: 0.063966, acc_cl:  18%] [G_gan loss: 3.413376, G_cl: 1.439177, recon: 0.164814] time: 1:28:16.935040 
*** TEST *** [D_gan accuracy : 0.484375 ] [D_cl accuracy : 0.46875 ]
[Epoch 17/170] [Batch 5/448] [D_gan loss: 0.126758, acc_gan:  96%] [D_cl loss: 0.055586, acc_cl:  20%] [G_gan loss: 3.552326, G_cl: 1.422259, recon: 0.168631] time: 1:28:21.083030 
[Epoch 17/170] [Batch 10/448] [D_gan loss: 0.096463, acc_gan:  96%] [D_cl loss: 0.049696, acc_cl:  19%] [G_gan loss: 3.959332, G_cl: 1.346498, recon: 0.157500] time: 1:28:24.482239 
[Epoch 17/170] [Batch 15/448] [D_gan loss: 0.082468, acc_gan:  97%] [D_cl loss: 0.051282, acc_cl:  23%] [G_gan loss: 4.577905, G_cl: 1.388296, recon: 0.172299] time: 1:28:27.832618 
[Epoch 17/170] [Batch 20/448] [D_gan loss: 0.064573, acc_gan:  97%] [D_cl loss: 0.052288, acc_cl:  18%] [G_gan loss: 4.774774, G_cl: 1.446055, recon: 0.173589] time: 1:28:31.106059 
[Epoch 17/170] [Batch 2

[Epoch 17/170] [Batch 225/448] [D_gan loss: 0.070334, acc_gan:  98%] [D_cl loss: 0.049893, acc_cl:  27%] [G_gan loss: 4.381474, G_cl: 1.409064, recon: 0.164412] time: 1:30:50.285819 
[Epoch 17/170] [Batch 230/448] [D_gan loss: 0.099636, acc_gan:  97%] [D_cl loss: 0.047280, acc_cl:  23%] [G_gan loss: 4.730514, G_cl: 1.490409, recon: 0.167832] time: 1:30:53.584085 
[Epoch 17/170] [Batch 235/448] [D_gan loss: 0.117752, acc_gan:  96%] [D_cl loss: 0.053120, acc_cl:  22%] [G_gan loss: 4.336647, G_cl: 1.290740, recon: 0.174435] time: 1:30:56.799225 
[Epoch 17/170] [Batch 240/448] [D_gan loss: 0.056309, acc_gan:  98%] [D_cl loss: 0.045317, acc_cl:  14%] [G_gan loss: 4.781244, G_cl: 1.368756, recon: 0.188630] time: 1:31:00.135375 
[Epoch 17/170] [Batch 245/448] [D_gan loss: 0.071173, acc_gan:  97%] [D_cl loss: 0.050781, acc_cl:  17%] [G_gan loss: 5.291028, G_cl: 1.416328, recon: 0.165059] time: 1:31:03.396855 
[Epoch 17/170] [Batch 250/448] [D_gan loss: 0.069639, acc_gan:  97%] [D_cl loss: 0.04

[Epoch 18/170] [Batch 0/448] [D_gan loss: 0.155617, acc_gan:  94%] [D_cl loss: 0.060679, acc_cl:  20%] [G_gan loss: 3.588289, G_cl: 1.388926, recon: 0.171495] time: 1:33:22.585407 
*** TEST *** [D_gan accuracy : 0.59375 ] [D_cl accuracy : 0.34375 ]
[Epoch 18/170] [Batch 5/448] [D_gan loss: 0.143947, acc_gan:  94%] [D_cl loss: 0.056013, acc_cl:  20%] [G_gan loss: 4.273084, G_cl: 1.369143, recon: 0.161783] time: 1:33:26.915982 
[Epoch 18/170] [Batch 10/448] [D_gan loss: 0.065992, acc_gan:  98%] [D_cl loss: 0.047533, acc_cl:  19%] [G_gan loss: 4.149911, G_cl: 1.582599, recon: 0.168120] time: 1:33:30.364571 
[Epoch 18/170] [Batch 15/448] [D_gan loss: 0.136520, acc_gan:  95%] [D_cl loss: 0.055197, acc_cl:  27%] [G_gan loss: 3.491170, G_cl: 1.377335, recon: 0.174770] time: 1:33:33.786924 
[Epoch 18/170] [Batch 20/448] [D_gan loss: 0.122399, acc_gan:  95%] [D_cl loss: 0.051313, acc_cl:  24%] [G_gan loss: 3.975821, G_cl: 1.349620, recon: 0.164566] time: 1:33:37.127209 
[Epoch 18/170] [Batch 25